In [10]:

pip install deepimpute


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag


  Using cached deepimpute-1.2.tar.gz (9.8 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


In [ ]:
import scanpy as sc
import os
from glob import glob
from deepimpute.multinet import MultiNet

# -----------------------------
# Input & Output directories
# -----------------------------
input_dir = "dropout_h5ad"
output_dir = "Imputed_DCA_h5ad"
os.makedirs(output_dir, exist_ok=True)

# -----------------------------
# Find all .h5ad files
# -----------------------------
files = glob(os.path.join(input_dir, "*.h5ad"))

# -----------------------------
# DeepImpute Imputation for each file
# -----------------------------
for f in files:
    print(f"Processing {f} ...")
    
    # Load missing dataset
    adata_missing = sc.read_h5ad(f)
    
    # Copy to keep original safe
    adata_deep = adata_missing.copy()
    
    # Convert AnnData to dataframe for DeepImpute
    df = adata_deep.to_df()
    
    # Run DeepImpute
    model = MultiNet()
    model.fit(df)
    imputed_df = model.predict(df)
    
    # Put back into AnnData
    adata_deep.X = imputed_df.values
    
    # Save with matching name
    fname = os.path.basename(f).replace("adata_dropout", "adata_deepimpute_imputed")
    outpath = os.path.join(output_dir, fname)
    adata_deep.write(outpath)
    
    print(f"Saved imputed file to: {outpath}")

print("All files processed with DeepImpute")


ModuleNotFoundError: No module named 'deepimpute'

In [11]:
pip install dca



  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl.metadata (1.9 kB)
  Using cached protobuf-3.19.6-cp310-cp310-win_amd64.whl.metadata (806 bytes)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp310-cp310-win_amd64.whl.metadata (14 kB)
   ---------------------------------------- 0.0/895.7 kB ? eta -:--:--
   ----------------------- ---------------- 524.3/895.7 kB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 895.7/895.7 kB 3.4 MB/s  0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------- ----------- 1.0/1.5 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 4.1 MB/s  0:00:00

  Attempting uninstall: protobuf

    Found existing installation: protobuf 3.20.3

    Uninstalling protobuf-3.20.3:

   ------------- -------------------------- 1/3 [protobuf]
   ------------- -------------------------- 1/3 [protobuf]
   ------------- -------------------------- 1/3 [protobuf]
   -

In [19]:
import scanpy as sc
import os
from glob import glob
from keras.losses import KLDivergence
from dca.api import dca

# -----------------------------
# Input & Output directories
# -----------------------------
input_dir = "dropout_h5ad"
output_dir = "Imputed_DCA_h5ad"
os.makedirs(output_dir, exist_ok=True)

# -----------------------------
# Find all .h5ad files
# -----------------------------
files = glob(os.path.join(input_dir, "*.h5ad"))

# -----------------------------
# DCA Imputation for each file
# -----------------------------
for f in files:
    print(f"Processing {f} ...")
    
    # Load missing dataset
    adata_missing = sc.read_h5ad(f)
    
    # Copy to keep original safe
    adata_dca = adata_missing.copy()
    
    # Run DCA imputation directly
    dca.dca(adata_dca)
    
    # Save with matching name
    fname = os.path.basename(f).replace("adata_dropout", "adata_dca_imputed")
    outpath = os.path.join(output_dir, fname)
    adata_dca.write(outpath)
    
    print(f"Saved imputed file to: {outpath}")

print("All files processed with DCA")


ImportError: cannot import name 'kullback_leibler_divergence' from 'keras.losses' (c:\Users\prath\Downloads\ScRnaseq_research\venv\lib\site-packages\keras\losses\__init__.py)

In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.Collecting torch
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/241.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.4 MB ? eta -:--:--
   ---------

In [ ]:
import os
from glob import glob
from pathlib import Path
import numpy as np
import scanpy as sc
import torch
import torch.nn as nn
import torch.optim as optim

# -----------------------------
# Config
# -----------------------------
INPUT_DIR = "dropout_h5ad"
OUTPUT_DIR = "Imputed_scGAN_h5ad"
os.makedirs(OUTPUT_DIR, exist_ok=True)

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Training hyperparameters
BATCH_SIZE = 256
EPOCHS = 100
LR_G = 1e-3
LR_D = 1e-3
HINT_RATE = 0.9     # fraction of mask entries revealed to the discriminator
ALPHA = 10.0        # weight on reconstruction loss for observed entries
HIDDEN = 512        # width of hidden layers (tweak for capacity/speed)

# -----------------------------
# Utilities
# -----------------------------
def to_tensor(x):
    if hasattr(x, "A"):  # sparse to dense if needed
        x = x.A
    return torch.tensor(x, dtype=torch.float32, device=DEVICE)

def make_mask(x):
    """
    Create mask of observed entries (1 = observed nonzero, 0 = zero/missing).
    Assumes 'dropouts' are zeros in X.
    """
    if hasattr(x, "A"):
        x = x.A
    return (x > 0).astype(np.float32)

def batch_iter(X, M, batch_size=BATCH_SIZE, shuffle=True):
    n = X.shape[0]
    idx = np.arange(n)
    if shuffle:
        np.random.shuffle(idx)
    for start in range(0, n, batch_size):
        sel = idx[start:start+batch_size]
        yield X[sel], M[sel]

# -----------------------------
# Models
# -----------------------------
class MLP(nn.Module):
    def __init__(self, in_dim, out_dim, hidden=HIDDEN):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim),
        )
    def forward(self, x):
        return self.net(x)

class ScGAIN:
    """
    GAIN-like imputation for scRNA-seq (cells x genes).
    Generator sees [X_tilde, M, Z], outputs X_hat; Discriminator sees [X_hat, H],
    predicts which entries are observed vs imputed.
    """
    def __init__(self, n_genes, hint_rate=HINT_RATE, alpha=ALPHA, lr_g=LR_G, lr_d=LR_D, hidden=HIDDEN):
        self.n_genes = n_genes
        self.hint_rate = hint_rate
        self.alpha = alpha

        # Generator input: X_tilde (filled with observed values where M=1, zeros where M=0),
        # noise Z for missing positions, and the mask M itself (optional but helpful).
        gen_in = n_genes * 2  # [X_tilde, M]
        self.G = MLP(gen_in, n_genes, hidden).to(DEVICE)

        # Discriminator input: concatenation of X_hat and hint matrix H (same size as M, partially revealed)
        disc_in = n_genes * 2  # [X_hat, H]
        self.D = MLP(disc_in, n_genes, hidden).to(DEVICE)

        self.opt_g = optim.Adam(self.G.parameters(), lr=lr_g)
        self.opt_d = optim.Adam(self.D.parameters(), lr=lr_d)
        self.bce = nn.BCEWithLogitsLoss(reduction="mean")
        self.mse = nn.MSELoss(reduction="mean")

    def _sample_Z(self, shape):
        # noise for missing entries
        return torch.rand(shape, device=DEVICE)

    def _sample_H(self, M):
        # hint matrix: with prob=hint_rate, reveal true mask entry; else 0.5 (unknown)
        B = torch.bernoulli(self.hint_rate * torch.ones_like(M))
        H = B * M + (1.0 - B) * 0.5
        return H

    def fit(self, X, M, epochs=EPOCHS, batch_size=BATCH_SIZE):
        self.G.train()
        self.D.train()
        n_cells, n_genes = X.shape

        for epoch in range(1, epochs + 1):
            g_losses, d_losses = [], []
            for Xb_np, Mb_np in batch_iter(X, M, batch_size=batch_size, shuffle=True):
                Xb = to_tensor(Xb_np)        # (B, G)
                Mb = to_tensor(Mb_np)        # (B, G)

                # Build X_tilde (observed kept, missing set to 0) and noise Z
                Zb = self._sample_Z(Xb.shape)
                X_tilde = Mb * Xb + (1.0 - Mb) * 0.0  # keep observed; zeros for missing
                G_in = torch.cat([X_tilde, Mb], dim=1)  # [X_tilde, M]
                X_hat = self.G(G_in)  # generator output (imputed full matrix)

                # Compose completed data: use observed from X, imputed from X_hat
                X_completed = Mb * Xb + (1.0 - Mb) * X_hat

                # ---- Train Discriminator ----
                H = self._sample_H(Mb)  # hint
                D_in = torch.cat([X_completed.detach(), H], dim=1)
                D_logits = self.D(D_in)

                # Discriminator target is the true mask M (observed=1, imputed=0)
                d_loss = self.bce(D_logits, Mb)
                self.opt_d.zero_grad()
                d_loss.backward()
                self.opt_d.step()

                # ---- Train Generator ----
                H = self._sample_H(Mb)
                D_in_g = torch.cat([X_completed, H], dim=1)
                D_logits_g = self.D(D_in_g)

                # Adversarial loss: fool D into predicting observed (1) on imputed entries
                g_adv_loss = self.bce(D_logits_g, torch.ones_like(Mb))

                # Reconstruction loss on observed entries only
                g_rec_loss = self.mse(Mb * X_hat, Mb * Xb)

                g_loss = g_adv_loss + self.alpha * g_rec_loss
                self.opt_g.zero_grad()
                g_loss.backward()
                self.opt_g.step()

                g_losses.append(g_loss.item())
                d_losses.append(d_loss.item())

            if epoch % 10 == 0 or epoch == 1:
                print(f"[Epoch {epoch:3d}/{epochs}]  G_loss={np.mean(g_losses):.4f}  D_loss={np.mean(d_losses):.4f}")

    @torch.no_grad()
    def impute(self, X, M):
        self.G.eval()
        X_torch = to_tensor(X)
        M_torch = to_tensor(M)
        X_tilde = M_torch * X_torch
        G_in = torch.cat([X_tilde, M_torch], dim=1)
        X_hat = self.G(G_in)
        X_completed = M_torch * X_torch + (1.0 - M_torch) * X_hat
        return X_completed.cpu().numpy()

# -----------------------------
# Runner over all .h5ad files
# -----------------------------
def run_scgan_on_folder(input_dir=INPUT_DIR, output_dir=OUTPUT_DIR,
                        epochs=EPOCHS, batch_size=BATCH_SIZE):
    files = sorted(glob(os.path.join(input_dir, "*.h5ad")))
    if not files:
        print(f"No .h5ad files found in {input_dir}")
        return

    for f in files:
        print(f"\n=== scGAN imputation on: {f} ===")
        adata = sc.read_h5ad(f)

        # Use the current .X (should contain dropout-simulated zeros)
        X = adata.X.A if hasattr(adata.X, "A") else adata.X
        X = X.astype(np.float32)
        # Mask: observed non-zero entries are 1; zeros are assumed missing
        M = make_mask(X)

        # Optional: per-gene scaling to stabilize training (undo after imputation)
        gene_mean = np.mean(X, axis=0, keepdims=True)
        gene_std = np.std(X, axis=0, keepdims=True) + 1e-6
        X_scaled = (X - gene_mean) / gene_std

        scgan = ScGAIN(n_genes=X.shape[1])
        scgan.fit(X_scaled, M, epochs=epochs, batch_size=batch_size)
        X_imputed_scaled = scgan.impute(X_scaled, M)

        # Unscale back
        X_imputed = X_imputed_scaled * gene_std + gene_mean

        # Keep observed entries exactly (do not alter true measurements)
        X_final = M * X + (1.0 - M) * X_imputed

        # Write back and save
        adata_out = adata.copy()
        adata_out.X = X_final
        out_name = os.path.basename(f).replace("adata_dropout", "adata_scGAN_imputed")
        out_path = os.path.join(output_dir, out_name)
        adata_out.write(out_path)
        print(f"✅ Saved scGAN-imputed file to: {out_path}")

# -----------------------------
# Run
# -----------------------------
if __name__ == "__main__":
    run_scgan_on_folder()
